In [1]:
import re
import pandas as pd
import numpy as np
import time
from collections import defaultdict

In [2]:
df_ads = pd.read_csv("YandexAdsad.csv", sep=';', usecols=[1], names=['text'])

twits1 = pd.read_csv('ptwits.csv', sep=';', usecols=[3], names=['text'])
twits2 = pd.read_csv('ntwits.csv', sep=';', usecols=[3], names=['text'])

df_twits = pd.concat([twits1, twits2])

In [3]:
df_twits

,text
0,"@first_timee хоть я и школота, но поверь, у на..."
1,"Да, все-таки он немного похож на него. Но мой ..."
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...
3,"RT @digger2912: ""Кто то в углу сидит и погибае..."
4,@irina_dyshkant Вот что значит страшилка :D\nН...
...,...
111918,Но не каждый хочет что то исправлять:( http://...
111919,скучаю так :-( только @taaannyaaa вправляет мо...
111920,"Вот и в школу, в говно это идти уже надо("
111921,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :..."


In [4]:
display(df_ads.head())
display(df_twits.head())

,text
0,Доставка по РФ! Доставка транспортной компание...
1,Наличие на складе! Цены и качество от официаль...
2,Доставка по РФ. Оплата при получении! Наличие ...
3,Наличие на складе! Цены и качество от официаль...
4,Buderus! Цены от официального дилера. Гарантия...


,text
0,"@first_timee хоть я и школота, но поверь, у на..."
1,"Да, все-таки он немного похож на него. Но мой ..."
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...
3,"RT @digger2912: ""Кто то в углу сидит и погибае..."
4,@irina_dyshkant Вот что значит страшилка :D\nН...


In [5]:
df_ads['target'] = 1
df_twits['target'] = 0

ads = df_ads[['text', 'target']]
twits = df_twits[['text', 'target']]

In [6]:
ads

,text,target
0,Доставка по РФ! Доставка транспортной компание...,1
1,Наличие на складе! Цены и качество от официаль...,1
2,Доставка по РФ. Оплата при получении! Наличие ...,1
3,Наличие на складе! Цены и качество от официаль...,1
4,Buderus! Цены от официального дилера. Гарантия...,1
...,...,...
2084087,Для Вашего принтера! Картридж XEROX 108r00909 ...,1
2084088,Для Вашего принтера! Картридж XEROX 3100 MFP к...,1
2084089,Для Вашего принтера! Картридж XEROX 3100MFP Га...,1
2084090,Для Вашего принтера! Картридж XEROX 3117 купит...,1


In [7]:
def preproc(s: str):
    def is_simple(word: str):
        return all((char.isalpha() for char in word))
    return ' '.join({word for word in s.lower().split() if len(word) > 3 and is_simple(word)})

ads['clear'] = ads['text'].apply(preproc)
twits['clear'] = twits['text'].apply(preproc)

In [8]:
!notify-send hello

In [10]:
df = pd.concat([ads, twits]).sample(frac=1).reset_index(drop=True)
df = df[df['clear'] != '']

In [11]:
len(df)

2291978

In [12]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.1, random_state=42)
print("TRAIN size:", len(df_train))
print("TEST size:", len(df_test))

TRAIN size: 2062780
TEST size: 229198


In [13]:
import gensim
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [14]:
w2v_model = gensim.models.word2vec.Word2Vec(size=1000, 
                                            window=10, 
                                            min_count=10, 
                                            workers=4)

In [15]:
w2v_model.build_vocab(df['clear'].apply(str.split))

2021-01-14 02:38:07,046 : INFO : collecting all words and their counts
2021-01-14 02:38:07,048 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-01-14 02:38:07,075 : INFO : PROGRESS: at sentence #10000, processed 41428 words, keeping 10772 word types
2021-01-14 02:38:07,100 : INFO : PROGRESS: at sentence #20000, processed 82750 words, keeping 16291 word types
2021-01-14 02:38:07,132 : INFO : PROGRESS: at sentence #30000, processed 124056 words, keeping 20620 word types
2021-01-14 02:38:07,161 : INFO : PROGRESS: at sentence #40000, processed 165347 words, keeping 24370 word types
2021-01-14 02:38:07,183 : INFO : PROGRESS: at sentence #50000, processed 206456 words, keeping 27635 word types
2021-01-14 02:38:07,209 : INFO : PROGRESS: at sentence #60000, processed 247664 words, keeping 30457 word types
2021-01-14 02:38:07,235 : INFO : PROGRESS: at sentence #70000, processed 289074 words, keeping 33244 word types
2021-01-14 02:38:07,259 : INFO : PROGRESS: at se

In [16]:
words = w2v_model.wv.vocab.keys()
vocab_size = len(words)
print("Vocab size", vocab_size)

Vocab size 36100


In [17]:
docs = df['clear'].apply(str.split)
w2v_model.train(docs, total_examples=len(docs), epochs=3)

2021-01-14 02:38:32,127 : INFO : training model with 4 workers on 36100 vocabulary and 1000 features, using sg=0 hs=0 sample=0.001 negative=5 window=10
2021-01-14 02:38:33,138 : INFO : EPOCH 1 - PROGRESS: at 2.75% examples, 220325 words/s, in_qsize 7, out_qsize 0
2021-01-14 02:38:34,147 : INFO : EPOCH 1 - PROGRESS: at 5.92% examples, 236489 words/s, in_qsize 6, out_qsize 1
2021-01-14 02:38:35,167 : INFO : EPOCH 1 - PROGRESS: at 9.19% examples, 243880 words/s, in_qsize 7, out_qsize 0
2021-01-14 02:38:36,184 : INFO : EPOCH 1 - PROGRESS: at 12.35% examples, 245575 words/s, in_qsize 7, out_qsize 0
2021-01-14 02:38:37,225 : INFO : EPOCH 1 - PROGRESS: at 15.63% examples, 247086 words/s, in_qsize 6, out_qsize 1
2021-01-14 02:38:38,249 : INFO : EPOCH 1 - PROGRESS: at 18.69% examples, 245950 words/s, in_qsize 7, out_qsize 0
2021-01-14 02:38:39,250 : INFO : EPOCH 1 - PROGRESS: at 21.97% examples, 248413 words/s, in_qsize 7, out_qsize 0
2021-01-14 02:38:40,250 : INFO : EPOCH 1 - PROGRESS: at 24.9

(24126718, 28399998)

In [19]:
w2v_model.most_similar(positive=['хорошо'])

<ipython-input-19-909e824b505e>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  w2v_model.most_similar(positive=['хорошо'])
2021-01-14 02:43:14,484 : INFO : precomputing L2-norms of word weight vectors


[('круто', 0.9079179763793945),
 ('видимо', 0.9067595601081848),
 ('пошли', 0.9044078588485718),
 ('немного', 0.9029951095581055),
 ('рада', 0.9016052484512329),
 ('обычно', 0.9011313915252686),
 ('думала', 0.9000774621963501),
 ('народ', 0.8977317810058594),
 ('ахаха', 0.8977190852165222),
 ('соскучилась', 0.8967077732086182)]

In [20]:
results = np.array([w2v_model[word] for word in preproc(test_text).split() if word in w2v_model.wv])

<ipython-input-20-ce7e928b58fa>:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  results = np.array([w2v_model[word] for word in preproc(test_text).split() if word in w2v_model.wv])


In [22]:
def chuncks(gen, chunck_size=100):
    iterator = iter(gen)
    flag = True
    while flag:
        res = []
        for _ in range(chunck_size):
            try:
                res.append(next(iterator)) 
            except StopIteration:
                flag = False
                break
        yield res   


for chunc in chuncks(range(20), 3):
    print(chunc)

[0, 1, 2]
[3, 4, 5]
[6, 7, 8]
[9, 10, 11]
[12, 13, 14]
[15, 16, 17]
[18, 19]


In [31]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(alpha=1, max_iter=100)

def get_text_vector(text: str):
    return sum(np.array([w2v_model.wv[word] if (word in w2v_model.wv) else np.zeros(1000) for word in text.split() ]))

def vectors_generator(texts):
    for text in texts:
        yield get_text_vector(text)

j = 0
for batch in chuncks(zip(vectors_generator(df['clear']), df['target']), 3000):
#     print([(len(item), target) for item, target in batch])
    mlp.partial_fit(*list(zip(*batch)), classes=[0,1])
    j += 1
    if j % 10 == 0: 
        print(f"{j}/{len(df)//3000} done...\r", end='')
    
# mlp.partial_fit()
# df_vec = pd.DataFrame(df[['clear', 'target']], columns=['clear', 'target', *list(w2v_model.wv.vocab.keys())])

# df_vec.head()

In [18]:
test_text = """Магазин «Шар идеальная форма» предлагает одежду больших размеров дамам, имеющим завидные формы. «Шар идеальная форма» является уникальной торговой площадкой с широким ассортиментом от производителей из разных стран. В магазине собраны модели от известных дизайнеров Турции, России и Беларуси. Магазин «Шар идеальная форма я» позволяет любой моднице получить желаемый товар по доступным ценам. Спешите оформить заказ, и менеджеры мигом доставят его вам. Для постоянных клиентов действует система скидок."""
test_text

'Магазин «Шар идеальная форма» предлагает одежду больших размеров дамам, имеющим завидные формы. «Шар идеальная форма» является уникальной торговой площадкой с широким ассортиментом от производителей из разных стран. В магазине собраны модели от известных дизайнеров Турции, России и Беларуси. Магазин «Шар идеальная форма я» позволяет любой моднице получить желаемый товар по доступным ценам. Спешите оформить заказ, и менеджеры мигом доставят его вам. Для постоянных клиентов действует система скидок.'

In [35]:
test_non_ad = """Будущий богослов родился в небольшом городке Аудеватер в южной части Испанских Нидерландов. Получив начальное образование на родине, продолжил изучать богословие в университетах Марбурга, Лейдена, Женевы и Базеля. В Женеве преподавателем Арминия стал известный протестантский богослов Теодор Беза, сподвижник Жана Кальвина. Полученные рекомендации позволили Арминию получить должность пастора в Амстердаме в 1588 году"""
test_non_ad

'Будущий богослов родился в небольшом городке Аудеватер в южной части Испанских Нидерландов. Получив начальное образование на родине, продолжил изучать богословие в университетах Марбурга, Лейдена, Женевы и Базеля. В Женеве преподавателем Арминия стал известный протестантский богослов Теодор Беза, сподвижник Жана Кальвина. Полученные рекомендации позволили Арминию получить должность пастора в Амстердаме в 1588 году'

In [38]:
test_non_ad2 = """Полагая, что Арминий является сторонником доктрины о свободе воли и деятельном участии человека в деле своего спасения, Гомарус обвинял его в ереси пелагианства и полупелагианства. Их конфликт неоднократно становился предметом разбирательства гражданских властей республики, и своё самое известное произведение, «Объявление мнения», Арминий представил 20 октября 1608 года в своей речи перед Генеральными штатами. Богослов скончался в разгар споров относительно своего учения, и его последователи, получившие название ремонстрантов, организационно оформились после его смерти. """
test_non_ad2

'Полагая, что Арминий является сторонником доктрины о свободе воли и деятельном участии человека в деле своего спасения, Гомарус обвинял его в ереси пелагианства и полупелагианства. Их конфликт неоднократно становился предметом разбирательства гражданских властей республики, и своё самое известное произведение, «Объявление мнения», Арминий представил 20 октября 1608 года в своей речи перед Генеральными штатами. Богослов скончался в разгар споров относительно своего учения, и его последователи, получившие название ремонстрантов, организационно оформились после его смерти. '

In [39]:
mlp.predict([get_text_vector(text) for text in (test_text, test_non_ad, test_non_ad2)])

array([1, 1, 0])

In [40]:
from joblib import dump, load
dump(mlp, "model.joblib")

['model.joblib']

In [41]:
mlp = load("model.joblib")

FileNotFoundError: [Errno 2] No such file or directory: 'model.joblib'